In [1]:
!git clone https://github.com/AleSilva04/wv72_tf_201822717_201620127_201816502_20181d073_201815776.git

Cloning into 'wv72_tf_201822717_201620127_201816502_20181d073_201815776'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 143 (delta 62), reused 21 (delta 7), pack-reused 0
Receiving objects: 100% (143/143), 285.05 KiB | 6.48 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import LeerCsv as lee
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Generar_Grafo as ga
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Funciones_Agrupar as ag

In [3]:
G = ga.generateGridGraph(1000)

In [4]:
almacen = lee.leer_archivo("wv72_tf_201822717_201620127_201816502_20181d073_201815776/dataset/warehouses.csv")
entrega = lee.leer_archivo("wv72_tf_201822717_201620127_201816502_20181d073_201815776/dataset/delivery-points.csv")

In [5]:
almacenes = []
for x,y in almacen:
  almacenes.append(y*1000+x)

ptsEntrega = []

for x,y in entrega:
  ptsEntrega.append(y*1000 +x)

In [6]:
#Le asignamos un almacen a cada uno de los 5000 puntos de entrega
def dist(a,b):
  x1,y1 = a
  x2,y2 = b
  return abs(x1-x2) + abs(y1-y2)

group =[-1]*5000
for i in range(5000):
  m=0
  d=dist(entrega[i],almacen[0])
  for j in range(1,100):
    d2 = dist(entrega[i],almacen[j])
    if d2 < d:
      d=d2
      m = j
  group[i]=m

In [7]:
#Agrupamos todos los nodos en su correspondiente grupo  
groupNodes = [[] for _ in range(100)]
for i in range(5000):
  g = group[i]
  groupNodes[g].append(ptsEntrega[i])

In [8]:
#Dividimos el Grafo General en subgrafos o regiones donde en cada una estan un almacen y sus respectivos puntos de entrega asignados
regions = [[] for _ in range(100)]
for i in range(100):
  g = groupNodes[i].copy()
  g.append(almacenes[i])
  limits = ag.limites(g)
  for y in range(limits[0],limits[1]+1):
    for x in range(limits[2],limits[3]+1):
      regions[i].append(y*1000+x)

In [9]:
import heapq as hq
import math
def dijkstra(M,Pts, G, s):
  visited = {node:False for node in M}
  visitedPts = []
  path = {node:None for node in M}
  cost = {node:math.inf for node in M}
  cost[s] = 0
  queue = [(0, s)]
  while queue:
    if len(visitedPts) == len(Pts):
      break
    g_u, u = hq.heappop(queue)
    if visited[u] is False:
      visited[u] = True
      if u in Pts:
        visitedPts.append(1)
      for v in G[u]:
        if v not in M:
          continue
        f = g_u + 1
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (f, v))

  return path, cost

In [ ]:
#Aplicamos dijkstra para cada region desde su respectivo almacen

paths_costs=[([],[]) for _ in range(100)]
for i in range(100):
  g=groupNodes[i].copy()
  g.append(almacenes[i])
  path, cost = dijkstra(regions[i],g,G,almacenes[i])
  paths_costs[i]=(path,cost)
  print("termina ",i)

In [40]:
def recorrido(f, parent):
  camino = [f]
  while f != None:
    a = parent[f]
    f = a
    camino.append(f)
  
  camino.pop(-1)
  return camino

def recorridos(groupNodes,paths_costs,s):
  parent, cost = paths_costs[s]
  destinos = groupNodes[s]
  caminos = []
  for destino in destinos:
    camino = recorrido(destino,parent)
    caminos.append(camino)
  return caminos #Se resto 1, porque agrego el vertice inicial y este no es un camino

In [41]:
def VRP(caminos, destinos):
  costo = 0
  rutas = []

  while destinos:
    max = -1
    cont = 0
    indice = -1
    encuentra = []

    for camino in caminos:
      n = len(camino)
      for i in range(n):
        if camino[i] in destinos:
          cont += 1
          encuentra.append(camino[i])
      if cont > max:
        max = cont
        destinos_llegados = encuentra
        caminito = camino
        
      cont = 0
      encuentra = []
    rutas.append(caminito)
    costo += (len(caminito)-1)

    #Eliminamos los destinos que encontramos para ya no buscarlos
    m = len(destinos)

    for dest in destinos_llegados:
      if dest in destinos:
        destinos.remove(dest)
    return rutas, costo

In [42]:
def VRP_total(groupNodes, paths_costs):
  for s in range(100): #Son 100 almacenes
    caminos = recorridos(groupNodes,paths_costs,s)
    rutas, costo = VRP(caminos,groupNodes[s].copy())
    print("Para el almacen ", s, ":")
    print(rutas)
    print("recorrido de: ",costo)
    print()

In [43]:
VRP_total(groupNodes, paths_costs)

Para el almacen  0 :
[[40949, 39949, 38949, 37949, 36949, 35949, 34949, 34948, 34947, 34946, 34945, 34944, 34943, 34942, 34941, 34940, 34939, 34938, 34937, 34936, 34935, 34934, 34933, 34932, 34931, 34930, 34929, 34928, 34927, 34926, 34925, 34924, 34923, 34922, 34921, 34920, 34919, 34918, 34917, 34916, 34915, 34914]]
recorrido de:  41

Para el almacen  1 :
[[834458, 833458, 832458, 831458, 830458, 829458, 828458, 827458, 826458, 825458, 824458, 823458, 822458, 821458, 820458, 819458, 818458, 817458, 816458, 815458, 814458, 814457, 814456, 814455, 814454, 814453, 814452, 814451, 814450, 814449, 814448, 814447, 814446, 814445, 814444, 814443, 814442, 814441, 814440, 814439, 814438, 814437, 814436, 814435, 814434, 814433, 814432, 814431, 814430, 814429, 814428, 814427, 814426, 814425, 814424, 814423]]
recorrido de:  55

Para el almacen  2 :
[[369943, 368943, 367943, 366943, 365943, 364943, 363943, 362943, 361943, 360943, 359943, 358943, 357943, 356943, 355943, 354943, 353943, 352943, 35194